In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.backends import cudnn

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
from typing import List, Any

# Set seed

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
cudnn.deterministic = True

# Get tokenizer

In [3]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


# Tokenization function

In [4]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

def tokenize_de(text: str) -> List[str]:
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    :param text: German string
    :return: list of tokens
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text: str) -> List[str]:
    """
    Tokenizes English text from a string into a list of strings (tokens) and reverses it
    :param text: English string
    :return: list of tokens
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

# Make object that will contain the tokenized text(including special tokens)

In [5]:
SRC = Field(tokenize = tokenize_de,
            init_token='<sos>',
            eos_token='<eos>',
            lower=True)

TRG = Field(tokenize=tokenize_en,
            init_token='<sos>',
            eos_token='<eos>',
            lower=True)

# Train, validation, test set split

In [6]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields= (SRC, TRG))

In [7]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


# Build vocab
Using the `min_freq` argument, we only allow tokens that appear at least 2 times to appear in our vocabulary. Tokens that appear once are converted to `<unk>`.
It is important to note that our vocabulary should only be built from the training set and not validation/test set. This prevents *information leakage* into our model, giving us artificially inflated validation/test scores.

In [8]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [9]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7853
Unique tokens in target (en) vocabulary: 5893


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Batching
We define the batch size and an iterator that will split each dataset into batches. This iterator should also be able to move the batches the designated device.

In [11]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device
)

# Seq2Seq Model
## Encoder

In [12]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src: [src len, batch size]

        embedded = self.dropout(self.embedding(src))
        #embedded: [src len, batch size, emb dim]

        outputs, (hidden, cell) = self.rnn(embedded)
        #outputs: [src len, batch size, hid dim * n directions]
        #hidden: [n layers * n directions, batch size, hid dim]
        #cell: [n layers * n directions, batch size, hid dim]

        #outputs are always from the top hidden layer

        return hidden, cell

## Decoder

In [13]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.linear = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input: [batch size] --> we will unsqueeze this vector to store the sentence
        # hidden: [n layers * n directions, batch size, hid dim]
        # cell: [n layers * n directions, batch size, hid dim]

        # Since n directions in the encoder will both always be 1,
        # hidden: [n layers, batch size, hid dim]
        # cell: [n layers, batch size, hid dim]

        input = input.unsqueeze(0)
        # input: [1, batch size]

        embedded = self.dropout(self.embedding(input))
        # embedded: [1, batch size, emb dim]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        #output: [seq len, batch size, hid dim * n directions]
        #hidden: [n layers * n directions, batch size, hid dim]
        #cell: [n layers * n directions, batch size, hid dim]

        #seq len and n directions will always be 1 in the decoder, therefore:
        #output: [1, batch size, hid dim]
        #hidden: [n layers, batch size, hid dim]
        #cell: [n layers, batch size, hid dim]

        prediction = self.linear(output.squeeze(0))
        # prediction: [batch size, output dim]

        return prediction, hidden, cell

## Model
\+ Teacher forcing \
**Note**: our decoder loop starts at 1, not 0. This means the 0th element of our `outputs` tensor remains all zeros. So our `trg` and `outputs` look something like:

$$\begin{align*}
\text{trg} = [<sos>, &y_1, y_2, y_3, <eos>]\\
\text{outputs} = [0, &\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

Later on when we calculate the loss, we cut off the first element of each tensor to get:

$$\begin{align*}
\text{trg} = [&y_1, y_2, y_3, <eos>]\\
\text{outputs} = [&\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

In [14]:
class Seq2Seq(nn.Module):
    def __init__(self,encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        # src: [src len, batch size]
        # tgt: [tgt len, batch size]

        batch_size = tgt.shape[1]
        tgt_len = tgt.shape[0]
        tgt_vocab_size = self.decoder.output_dim

        # tensor to store decoder output
        outputs = torch.zeros(tgt_len, batch_size, tgt_vocab_size).to(self.device)

        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)

        #first input of the decoder is used as the initial hidden state of the decoder
        input = tgt[0,:]

        for t in range(1, tgt_len):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output , hidden, cell = self.decoder(input, hidden, cell)

            # place predictions in a tensor holding predictions for each token
            outputs[t] = output

            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            # get the highest predicted token from our predictions
            top1 = output.argmax(1)

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = tgt[t] if teacher_force else top1

            return outputs

# Training

In [15]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

# Initialize Weights

In [16]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (linear): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Count trainable parameters in the model

In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 13,898,501 trainable parameters


# Optimizer

In [18]:
optimizer = optim.Adam(model.parameters())

# Loss Function

In [19]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index= TRG_PAD_IDX)

# Prepare Tensorboard

In [30]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir='runs/Seq2Seq_exp')

# Training

In [27]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()

    epoch_loss = 0
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)
        # trg: [tgt len, batch size]
        # output: [tgt len, batch size, output dim]

        output_dim = output.shape[-1]

        output = output[1:].view(-1, output_dim) # Exclude <sos>
        trg = trg[1:].view(-1) # Exclude <sos>
        #output: [(tgt len - 1) * batch size, output dim]
        #trg: [(tgt len - 1) * batch size]

        loss = criterion(output, trg)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

# Evaluate
We must remember to set the model to evaluation mode with `model.eval()`. This will turn off dropout (and batch normalization, if used). \
\
We use the with `torch.no_grad()` block to ensure no gradients are calculated within the block. This reduces memory consumption and speeds things up.

In [28]:
def evaluate(model, iterator, criterion):
    model.eval()

    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) # turn off teacher forcing
            # trg: [tgt len, batch size]
            # output: [tgt len, batch size, output dim]

            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [24]:
%load_ext tensorboard
%tensorboard --logdir runs --port=6006

Launching TensorBoard...

In [34]:
N_EPOCHS = 30
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    writer.add_scalar('training loss',train_loss,epoch)
    writer.add_scalar('validation loss',valid_loss,epoch)

writer.flush()
writer.close()

In [33]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test loss: 8.192 | Test PPL: 3611.003 |
